In [2]:
from Tkinter import *
from __future__ import division
import tkFileDialog
from math import cos, pi, fabs
from numpy import sign,arange, sin
import sys
from PIL import Image, ImageTk
from boundaries import boundaries
from walls import walls
from Sphero import Sphero
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg

# Addition of a Next button in the GUI window
def start():   # For the next button
#     fp = open("robot_trajectory3.txt","w")
    global pose, control, constraints, loop, radius, wall_width, world_size, flag
    scale = 520
    loop = not(loop)
    i=0
    while loop and i<=50:
        print "Entered"
        posep = pose
        print "Prediction to start with pose",pose
        pose = s.motion(pose,control,0.01)
        print "Prediction is completed",pose
        print "Check collision"
        f=s.draw(pose)
        pose = s.check_collision(constraints, pose, posep, flag)
        print "Collision detection is completed",pose
        f=s.draw(pose[0])
        robot_map(0.025+(pose[0][0])/scale,0.9-(pose[0][1])/scale,pose[1],radius/scale,wall_width/scale)
        root.update()
        print "Tada {0} \t {1} \t {2} \t{3}".format(pose[0][0],pose[0][1],pose[0][2],pose[1])
        pose = pose[0]
        i = i+1

def stop():
    global loop
    loop = not(loop)
    
def boost():
    global control
    control[0] = control[0] + 50
    if control[0] <= 0:
        control[0] =0

def slow():
    global control
    control[0] = control[0] - 50
    if control[0] <= 0:
        control[0] =0

def correct_wall():
    global pose, control, loop, flag
    loop = not(loop)
    pose = [100,150,pi/4]
    control = [100,0]
    loop = 0
    flag = 1
        
def robot_map(cx,cy,wall_type,rad,ww):
    if wall_type == 1:
        ax1 = fig.add_axes([cx-rad-ww, cy, ww, ww*3]) # GUI rectangle creation
        ax1.set_axis_off()
        ax2 = fig.add_axes([cx-rad-ww, cy-ww*3, ww, ww*3])
        ax2.set_axis_off()
        cmap1 = mpl.cm.gray
        cmap2 = mpl.cm.Greys
        norm = mpl.colors.Normalize()
        cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                       norm=norm,
                                       orientation='vertical')
        cb1.outline.set_visible(False)
        cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap2,
                                       norm=norm,
                                       orientation='vertical')
        cb1.outline.set_visible(False)
        plt.gcf().canvas.draw()
        
    elif wall_type == 2:
        ax1 = fig.add_axes([cx, cy-rad, ww*3, ww]) # GUI rectangle creation
        ax1.set_axis_off()
        ax2 = fig.add_axes([cx-ww*3, cy-rad, ww*3, ww])
        ax2.set_axis_off()
        cmap1 = mpl.cm.gray
        cmap2 = mpl.cm.Greys
        norm = mpl.colors.Normalize()
        cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                       norm=norm,
                                       orientation='horizontal')
        cb1.outline.set_visible(False)
        cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap2,
                                       norm=norm,
                                       orientation='horizontal')
        cb1.outline.set_visible(False)
        plt.gcf().canvas.draw()
        
    elif wall_type == 3:
        ax1 = fig.add_axes([cx+rad, cy, ww, ww*3]) # GUI rectangle creation
        ax1.set_axis_off()
        ax2 = fig.add_axes([cx+rad, cy-ww*3, ww, ww*3])
        ax2.set_axis_off()
        cmap1 = mpl.cm.gray
        cmap2 = mpl.cm.Greys
        norm = mpl.colors.Normalize()
        cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                       norm=norm,
                                       orientation='vertical')
        cb1.outline.set_visible(False)
        cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap2,
                                       norm=norm,
                                       orientation='vertical')
        cb1.outline.set_visible(False)
        plt.gcf().canvas.draw()
        
    elif wall_type == 4:
        ax1 = fig.add_axes([cx, cy+rad+ww, ww*3, ww]) # GUI rectangle creation
        ax1.set_axis_off()
        ax2 = fig.add_axes([cx-ww*3, cy+rad+ww, ww*3, ww])
        ax2.set_axis_off()
        cmap1 = mpl.cm.gray
        cmap2 = mpl.cm.Greys
        norm = mpl.colors.Normalize()
        cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap1,
                                       norm=norm,
                                       orientation='horizontal')
        cb1.outline.set_visible(False)
        cb1 = mpl.colorbar.ColorbarBase(ax2, cmap=cmap2,
                                       norm=norm,
                                       orientation='horizontal')
        cb1.outline.set_visible(False)
        plt.gcf().canvas.draw()
        
    else:
        pass
        
if __name__ =="__main__":

    # Specifications of the world
    origin = (2,2)
    world_size = (498, 498)   # 750,750
    wall_width = 10
    radius = 10
    plotShift = 0
    flag = 0 # For wall correction
    wall_location =[[origin[0]+(world_size[0])/4,origin[1]+wall_width,origin[0]+(world_size[0])/4,origin[1]+wall_width+(world_size[1])*3/4],
                    [origin[0]+(world_size[0])/2,origin[1]+wall_width+(world_size[1])*1/4,origin[0]+(world_size[0])/2,origin[1]-wall_width+world_size[1]],
                    [origin[0]+(world_size[0])*3/4,origin[1]+wall_width,origin[0]+(world_size[0])*3/4,origin[1]+wall_width+(world_size[1])*3/4]] 
    
    #Initialization of boundary and walls
    sri = boundaries(world_size,wall_width) 
    mw = walls(wall_width,wall_location)
    
    # Intialization of GUI
    root = Tk()
    root.title("Ball in a maze")
    frame1 = Frame(root)
    frame1.pack()
    w = Canvas(frame1,width=500,height=500,bg="white")
    w.pack(side=LEFT)
    
    
    # Robot map
    r = Canvas(frame1,width=500,height=500,bg="white")
    r.pack(side=RIGHT)
    fig = plt.figure(figsize=(6.15,6.15),facecolor='w',edgecolor='w')
    canvas = FigureCanvasTkAgg(fig, master=r)
    canvas.get_tk_widget().pack(side=RIGHT)
    
    
    # Creating world objects
    sri.draw(origin,w)
    mw.draw(wall_width,wall_location,w)
    parray = sri.boundary_grid(origin,w)
    warray = mw.wall_grid(wall_width,wall_location,w)
    
    # Creating Sphero robot
    s = Sphero(w)
    constraints = s.sphero_constraint(origin,wall_width,world_size,wall_location)
    
    pose = [100,150,pi/4]
    control = [100,0]
    loop = 0
    frame2 = Frame(root)
    frame2.pack()
    start = Button(frame2,text="START", width=10,command=start)
    start.grid(row=0,column=0)
    stop = Button(frame2,text="STOP",width=10,command=stop)
    stop.grid(row=0,column=1)
    boost = Button(frame2,text="BOOST", width=10,command=boost)
    boost.grid(row=0,column=2)
    slow = Button(frame2,text="SLOW", width=10,command=slow)
    slow.grid(row=0,column=3)
    wc = Button(frame2,text="Wall Correction", width=15,command=correct_wall)
    wc.grid(row=0,column=4)
    
    # Draw GUI
    root.mainloop()

Entered
Prediction to start with pose [100, 150, 0.7853981633974483]
Prediction is completed [103.18198051533946, 153.18198051533946, 0.7853981633974483]
Check collision
Collision detection is completed ([103.18198051533946, 153.18198051533946, 0.7853981633974483], 0, [0, 0, 0])
Tada 103.181980515 	 153.181980515 	 0.785398163397 	0
Entered
Prediction to start with pose [103.18198051533946, 153.18198051533946, 0.7853981633974483]
Prediction is completed [106.36396103067892, 156.36396103067892, 0.7853981633974483]
Check collision
Collision detection is completed ([106.36396103067892, 156.36396103067892, 0.7853981633974483], 0, [0, 0, 0])
Tada 106.363961031 	 156.363961031 	 0.785398163397 	0
Entered
Prediction to start with pose [106.36396103067892, 156.36396103067892, 0.7853981633974483]
Prediction is completed [109.54594154601838, 159.54594154601838, 0.7853981633974483]
Check collision
Collision detection is completed ([109.54594154601838, 159.54594154601838, 0.7853981633974483], 0, [

In [ ]:
import numpy as np
f = np.loadtxt(r"C:\Users\localadmin.TUD0035314\Documents\GitHub\New_world\robot_trajectory.txt")
print f[0:100]